In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dataset-age:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4784648%2F8101805%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240428%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240428T234913Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38925558c105126e7ef9ead3e9ae75f86702f39ddaddd5b2599ec7f34beccdba49feec5f10bd6db2bbfc615fa96cd303e5f4cd54a8516e13849a63ed49a2b213c710e82712ab22686e0683fa20ec0d9684aff69f2c588c05bbb1da3bfe48fd53aee94ca0107b97aafa6852a9f3c25f560cb2a01715951d8d1a53b040d2db76f342af8764e835c709b7586b256c26a461d22a334b66814d909a0e4340a7280af4086dc9c731193e7250be9647180e4b5863581936190ce01c40904d9ca849d234c7f2e39c328b048e8ea50521c8d8ba7470b1a7fb644c244a4b0511b8ba9e52b52bbb24ba15a535f1aaea6931d2518f7c8c8c880df804da30fa0b5a7c46d45929,age-metadata:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4784789%2F8102004%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240428%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240428T234913Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D54002e082713f571979dbd7b23ab0b92f3014255eb8a5e7079a025886b2896c070829843354a1fc1fd35262929624cc3c0819ed1237b82dcd5d610a5a40253079e34c83166655f6c4e2ea8eab9551779df038b4156f254945b22d76da776b7876ff17db34eb9de5cea3ebafa117fc9146c5a13070e4dc5454afcce65ff3a9234bcc87f4e84bf16fdc8c93f7ea088a8924f68908f6e5aba8347a0c066948d4641f6783d1b86f631db7edfb59f5334ef754192d0f6ddede9d11b5062e483f07237598e8e206a6103eda1f0979deea008d21c22a10b6a6d5defb9439c604432e3db75b5ded460688aa7dcfb737f665008cd8404f9658760b9e4361fe5c044009784,age-testdata:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4865651%2F8210545%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240428%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240428T234913Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D94803f1d17c303bf591b6fc622b4701fa960df3464eeff5ec70755af78529130d5a95bd764b9415223d2c891387043cc2a93d32ea3b6c4d63dbf7faf0da53d918de6354ebeffbdd3c32074b396fccffc920e9cfe12a78965a2c41d0a91042d43e233633252c9a7c25934d558c1cd33f3d819f8f01c9a2b89947a4aead9106c59f9e6d030dee47e298265478a71bf8efe4f79dfc1ee92a7139777fdaaca775f94053ca659e8572201c82569a040d2f1d00b29e7c1dd16f9bbb5be7fd770c52075c06f0ad2040be95dfc89ad6638cb25d2e49975dfd78e2eee53be84c21e0dbd083897388aa3dd55c41cace6952622ded8bbd23433faaa3cb4643a1daee2cf8996,age-testimages:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4865659%2F8210555%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240428%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240428T234913Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D494dd66dd63fa8bd28b3c9e4354077fa8aab87f2210cbc0c9040312b4821bdfcdd01f2fb5d9993e58eab7d942eb6348e4da78e4e4e1d40687ac55f0f8c1654823a40faa45284e26b923fa0209162e31c4bb923c83467e8d9677d4e3973851ff937c27218b07409df5af8188862833ead2d4baa715d63005c258d3e5ab11e179bc8e252bd090f13f319c593546b9712eed1420dab8659cf64188d25f3db150290f136974de5dd6032e5c702c4de5c5e13314c5b492202ceee818008bab872815cf11776dbd9cdc86ab3a4e5ba6ddaa701e0505c7d7835041e719d21c8680cfb173bac835656ed97b40bced93a75704438503c358fea4b2c34011c68cdb553a16e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 319179331 bytes downloaded
Downloaded and uncompressed: dataset-age
[==================================================] 3804893 bytes downloaded
Downloaded and uncompressed: age-metadata
[==================================================] 23056 bytes downloaded
Downloaded and uncompressed: age-testdata
[==================================================] 9144495 bytes downloaded
Downloaded and uncompressed: age-testimages
Data source import complete.


In [2]:
import cv2
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input,InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.losses import Huber


In [3]:
def load_and_filter_images(image_paths, base_directory):
    valid_images = []
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    for img_path in image_paths:
        full_path = os.path.join(base_directory, img_path)  # Correctly join the base directory with the image path
        image = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue
        faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
        if len(faces) == 1:
            valid_images.append(img_path)
    return valid_images

In [4]:
data = pd.read_csv('/kaggle/input/age-metadata/wiki_labels.csv')
data['full_path'] = data['full_path'].str.extract(r"(\d+\/[^']+\.png)")  # Regex to extract the relative path


In [5]:
base_directory = '/kaggle/input/dataset-age/wiki_labeled/wiki_labeled/'

In [6]:
valid_image_paths = load_and_filter_images(data['full_path'].dropna().tolist(), base_directory)
data = data[data['full_path'].isin(valid_image_paths)]

In [7]:
data['face_score'] = pd.to_numeric(data['face_score'], errors='coerce')
data = data[data['face_score'].notna() & (data['face_score'] != float('inf')) & data['second_face_score'].isna()]

In [8]:
base_year = 1.0
data['approx_dob_year'] = base_year + (data['dob'] - 366) / 365.25
data['age'] = data['photo_taken'] - data['approx_dob_year']
data['age'] = data['age'].round().astype('float64')
data = data[(data['age'] > 0) & (data['age'] <= 100)]


In [9]:
data['full_path'] = data['full_path'].str.extract(r"(\d+\/[^']+\.png)")

In [10]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.5, 1.5],
    validation_split=0.2
)


In [11]:
train_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory=base_directory,
    x_col='full_path',
    y_col='age',
    target_size=(100, 100),
    batch_size=32,
    color_mode='grayscale',
    class_mode='raw',
    subset='training'
)

Found 22836 validated image filenames.


In [12]:
validation_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory=base_directory,
    x_col='full_path',
    y_col='age',
    target_size=(100, 100),
    batch_size=32,
    color_mode='grayscale',
    class_mode='raw',
    subset='validation'
)


Found 5709 validated image filenames.


In [13]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import MeanAbsoluteError
model = Sequential([
    InputLayer(input_shape=(100, 100, 1)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1)
])

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=Adam(learning_rate=0.0001), loss=Huber(),  metrics=[MeanAbsoluteError()])

In [14]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50
713/713 [==============================] - 89s 113ms/step - loss: 15.9908 - mean_absolute_error: 13.9543 - val_loss: 12.7492 - val_mean_absolute_error: 12.6072 - lr: 1.0000e-04
Epoch 2/50
713/713 [==============================] - 79s 110ms/step - loss: 12.8529 - mean_absolute_error: 12.9211 - val_loss: 12.4630 - val_mean_absolute_error: 12.6333 - lr: 1.0000e-04
Epoch 3/50
713/713 [==============================] - 79s 111ms/step - loss: 12.4614 - mean_absolute_error: 12.6652 - val_loss: 12.2073 - val_mean_absolute_error: 12.4273 - lr: 1.0000e-04
Epoch 4/50
713/713 [==============================] - 69s 97ms/step - loss: 12.2781 - mean_absolute_error: 12.5037 - val_loss: 11.8848 - val_mean_absolute_error: 12.1047 - lr: 1.0000e-04
Epoch 5/50
713/713 [==============================] - 70s 98ms/step - loss: 12.0805 - mean_absolute_error: 12.3033 - val_loss: 11.7030 - val_mean_absolute_error: 11.9085 - lr: 1.0000e-04
Epoch 6/50
713/713 [==============================] - 78s 110m

In [15]:

judge_datagen = ImageDataGenerator(rescale=1./255)

judge_data = pd.read_csv('/kaggle/input/age-testdata/wiki_judge.csv')




In [16]:
judge_data['full_path'] = judge_data['full_path'].str.extract(r"(\d+\.png)")

In [17]:
judge_generator = judge_datagen.flow_from_dataframe(
    dataframe=judge_data,
    directory='/kaggle/input/age-testimages/wiki_judge_images/wiki_judge_images/',
    x_col='full_path',
    y_col=None,
    target_size=(100, 100),
    batch_size=32,
    color_mode='grayscale',
    class_mode=None,
    shuffle=False
)

Found 1409 validated image filenames.


In [18]:
import numpy as np
predicted_ages = model.predict(judge_generator)
predicted_ages = predicted_ages.flatten()

predicted_ages = np.clip(predicted_ages, 0, 100)




45/45 [==============================] - 1s 28ms/step


In [19]:
submission_df = pd.DataFrame({
    'ID': judge_data['ID'],
    'age': predicted_ages
})

submission_df.to_csv('/content/sample_data/submissionhuber1.csv', index=False)
